In [10]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt

# from MCForecastTools import MCSimulation
from MCForecastTools import MCSimulation

%matplotlib inline

ModuleNotFoundError: No module named 'MCForecastTools'

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

In [13]:
# Tickers
ticker = ["BABA", "DIS", "TSLA", "ETHE", "SPY"]

# set timeframe to '1D'

timeframe = "1D"


# Set start and end datetimes of 5 years
start_date = pd.Timestamp("2016-05-24", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-05-24", tz="America/New_York").isoformat()


# Get 5 year's worth of historical data 
all_stocks= alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# # Display sample data
all_stocks.head(10)

NameError: name 'alpaca' is not defined

In [14]:
all_stocks_df = all_stocks.dropna()

NameError: name 'all_stocks' is not defined

In [15]:
# Create and empty DataFrame for closing prices
all_closing_prices = pd.DataFrame()

# Fetch the closing prices 
all_closing_prices["BABA"] = all_stocks_df["BAB"]["close"]
all_closing_prices["SPY"] = all_stocks_df["SPY"]["close"]
all_closing_prices["DIS"] = all_stocks_df["DIS"]["close"]
all_closing_prices["ETHE"] = all_stocks_df["ETHE"]["close"]

# Drop the time component of the date
all_closing_prices.index = all_closing_prices.index.date

# Compute daily returns
all_returns = all_closing_prices.pct_change().dropna()

NameError: name 'all_stocks_df' is not defined

In [17]:
# Calculate Cumulative Returns
all_cumulative_returns = (1 + all_returns).cumprod()

# Plot Cummulative Returns
all_cumulative_returns.plot(title = 'Cumulative Returns', figsize = (20,10) )

NameError: name 'all_returns' is not defined

In [18]:
all_returns.plot.density()

NameError: name 'all_returns' is not defined

In [19]:
# Calculate Standard Deviation
all_std = all_returns.std()

# Calculate Annual Standard Deviation
all_annual_std = all_std * np.sqrt(252)

# Display Standard Deviation
all_annual_std

NameError: name 'all_returns' is not defined

In [20]:
# Calculate Correlation
all_correlation = all_returns.corr()

NameError: name 'all_returns' is not defined

In [22]:
# Plot Correlation 
sns.heatmap(all_correlation, vmin = -1, vmax = 1, annot = True)

NameError: name 'all_correlation' is not defined

In [23]:
# Calculate Covariance
baba_spy_covar = all_returns['BABA'].rolling(window = 60).cov(all_returns['SPY'])
dis_spy_covar = all_returns['DIS'].rolling(window = 60).cov(all_returns['SPY'])
tsla_spy_covar = all_returns['TSLA'].rolling(window = 60).cov(all_returns['SPY'])
ethe_spy_covar = all_returns['ETHE'].rolling(window = 60).cov(all_returns['SPY'])

NameError: name 'all_returns' is not defined

In [24]:
# Calculate Variance
baba_spy_var = all_returns['SPY'].rolling(window = 60).var()
dis_spy_var = all_returns['SPY'].rolling(window = 60).var()
tsla_spy_var = all_returns['SPY'].rolling(window = 60).var()
ethe_spy_var = all_returns['SPY'].rolling(window = 60).var()

NameError: name 'all_returns' is not defined

In [26]:
# Calculate Beta
baba_beta = baba_spy_covar / baba_spy_var
dis_beta = dis_spy_covar / dis_spy_var
tsla_beta = tsla_spy_covar / tsla_spy_var
ethe_beta = ethe_spy_covar / ethe_spy_var
combined_beta = baba_beta + dis_beta + tsla_beta +ethe_beta

NameError: name 'baba_spy_covar' is not defined

In [27]:
# Plot Beta
beta_plot = combined_beta.plot(title= "60 Day Beta", figsize=(20, 10))
bab_beta_plot.legend(["BABA", "DIS", "TSLA", "ETHE"])

NameError: name 'combined_beta' is not defined

In [28]:
# Calculate the Sharpe Ratio
all_sharpe_ratios = (all_returns.mean() * 252) / (all_returns.std() * np.sqrt(252))


NameError: name 'all_returns' is not defined

In [29]:
# Plot Sharpe Ratio
all_sharpe_ratios.plot(kind = "bar", title = "Sharpe Ratio")

NameError: name 'all_sharpe_ratios' is not defined